In [2]:
import torch

In [7]:
# SELF ATTENTION
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

head_size = 16
key = torch.nn.Linear(C, head_size, bias=False)
query = torch.nn.Linear(C, head_size, bias=False)
value = torch.nn.Linear(C, head_size, bias=False)
k = key(x) # B, T, head_size
q = query(x) # B, T, head_size
wei = q @ k.transpose(-2,-1) # only transpose last two dimensions: B,T,16 @ B,16,T ---> B,T,T
wei = wei * head_size ** -0.5 # we do this to prevent softmax from converging to a one-hot vector (control the variance at initialization)

tril = torch.tril(torch.ones(T,T)) # mask to stop future from communicating with past
wei = wei.masked_fill(tril == 0, float('-inf')) # DELETE THIS FOR ENCODER BLOCKS
wei = torch.nn.functional.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

# FOR ENCODER-DECODER ATTENTION, ONLY QUERIES ARE PRODUCED FROM X, & KEYS AND VALUES ARE FROM ENCODER